In [20]:
# !python -m spacy download en_core_web_sm
# !pip install rapidfuzz


In [21]:
import pandas as pd
import spacy
from collections import Counter, defaultdict
from rapidfuzz import fuzz

# -------------------- 1. Load Final Labelled Queries -------------------- #
def load_final_labelled_queries(filepath):
    df = pd.read_csv(filepath)
    if 'cleaned_query' not in df.columns or 'topic_label_final' not in df.columns:
        raise ValueError("Missing required columns in uploaded file.")
    return df

# -------------------- 2. Extract Entities -------------------- #
def extract_entities(df):
    nlp = spacy.load("en_core_web_sm")

    # Define domain-specific keyword buckets
    CUSTOM_BUCKETS = {
    "payment_method": ["google pay","card","credit card", "debit card", "paypal", "net banking", "upi", "wallet", "cash", "cash on delivery", "bank transfer"],
    "delivery_option": ["standard delivery", "express delivery", "pickup", "home delivery", "delivery time", "estimated delivery", "eta", "shipping method"],
    "delivery_address": ["shipping address", "delivery address", "new address", "change address", "set address"],
    "order_reference": ["cancel order", "order status", "track order", "modify order", "order number"],
    "invoice_reference": ["invoice", "invoice number", "last invoice", "check invoice"],
    "account_action": ["account deletion", "account recovery", "account creation", "edit profile", "reset password", "register", "sign up"],
    "refund_request": ["refund request", "refund policy", "cancellation policy", "cancelled", "cancellation fee"],
    "support_request": ["help me", "need help", "assistance", "can you help", "help to", "you help"],
    "status_check": ["check", "to check", "need to check", "check status", "check refund", "check order"],
    "recipient_person": ["mom", "wife", "dad", "daughter"], 
    "support_channel": ["agent", "customer support", "live chat", "human agent", "talk to human"],
    "newsletter_action": ["newsletter", "subscribe", "unsubscribe", "mailing list"],
    "purchase_help": ["buy", "purchase", "payment failed", "transaction error", "cannot pay"],
    "instruction_request": ["how to", "know how", "can i", "could you", "know what"],
    "general_complaint": ["problem", "issue", "report", "complaint", "feedback", "claim", "escalate"],
}


    FUZZY_MATCH_THRESHOLD = 90
    ent_counter = Counter()
    ent_to_rows = defaultdict(set)
    custom_counter = defaultdict(Counter)
    custom_to_rows = defaultdict(lambda: defaultdict(set))
    rows_with_entity = set()
    entity_list = set()
    entity_type_list = set()

    for idx, query in enumerate(df['cleaned_query']):
        query_lower = query.lower()
        doc = nlp(query_lower)
        found_entity = False

        for ent in doc.ents:
            key = (ent.label_, ent.text.lower())
            ent_counter[key] += 1
            ent_to_rows[key].add(idx + 2)
            entity_list.add(ent.text.lower())
            entity_type_list.add(ent.label_)
            found_entity = True

        for bucket, keywords in CUSTOM_BUCKETS.items():
            for kw in keywords:
                score = fuzz.partial_ratio(query_lower, kw.lower())
                if score >= FUZZY_MATCH_THRESHOLD:
                    custom_counter[bucket][kw.lower()] += 1
                    custom_to_rows[bucket][kw.lower()].add(idx + 2)
                    entity_list.add(kw.lower())
                    entity_type_list.add(bucket)
                    found_entity = True

        if found_entity:
            rows_with_entity.add(idx)

    # Convert counters to DataFrames
    spacy_df = pd.DataFrame([
        (etype, evalue, count, sorted(list(ent_to_rows[(etype, evalue)])))
        for (etype, evalue), count in ent_counter.items()
    ], columns=["Entity Type", "Entity Value", "Frequency", "Query Rows"])

    custom_dfs = []
    for bucket, c in custom_counter.items():
        df_bucket = pd.DataFrame([
            (bucket, val, count, sorted(list(custom_to_rows[bucket][val])))
            for val, count in c.items()
        ], columns=["Entity Type", "Entity Value", "Frequency", "Query Rows"])
        custom_dfs.append(df_bucket)

    full_entity_df = pd.concat([spacy_df] + custom_dfs, ignore_index=True)
    full_entity_df = full_entity_df.sort_values("Frequency", ascending=False)
    full_entity_df.to_csv("identified_entities.csv", index=False)

    # Save rows with no entities
    all_indices = set(range(len(df)))
    no_entity_indices = sorted(list(all_indices - rows_with_entity))
    no_entity_df = df.iloc[no_entity_indices]
    no_entity_df.to_csv("queries_with_no_entities.csv", index=False)

    print("\n🔍 List of unique extracted entity values:")
    print(sorted(entity_list))
    print("\n📘 List of unique entity types:")
    print(sorted(entity_type_list))

    return full_entity_df

# -------------------- 3. Run Full Pipeline -------------------- #
def run_step_4_pipeline():
    df = load_final_labelled_queries("clustered_labelled_queries_final.csv")
    entity_df = extract_entities(df)
    print("\n✅ Entities extracted and saved to identified_entities.csv")
    print("📄 Queries with no entities saved to queries_with_no_entities.csv")
    return entity_df

# -------------------- 4. Execute -------------------- #
if __name__ == "__main__":
    entity_df = run_step_4_pipeline()



🔍 List of unique extracted entity values:
['1', '1 months ago', '1 purchases ago', '10 months ago', '10 purchases ago', '11', '11 months ago', '2', '2 months ago', '3', '3 purchases ago', '4 purchases', '5', '6 months ago', '6 purchases', '7 months ago', '8 months ago', '9 months ago', '9 purchases ago', 'account deletion', 'agent', 'april', 'assistance', 'august', 'buy', 'can i', 'can you help', 'cancel order', 'cancellation fee', 'cannot pay', 'card', 'check', 'check invoice', 'check order', 'check refund', 'check status', 'claim', 'complaint', 'could you', 'customer support', 'dad', 'daughter', 'december', 'delivery address', 'eight months ago', 'eleven months ago', 'eta', 'february', 'feedback', 'five purchases ago', 'four months ago', 'google pay', 'help me', 'help to', 'home delivery', 'hours', 'how to', 'human agent', 'invoice', 'issue', 'january', 'july', 'june', 'know how', 'know what', 'last invoice', 'last month', 'mom', 'need help', 'need to check', 'new address', 'newslet

In [22]:
print("\n📊 Entity DataFrame:")
entity_df  # Display first 10 rows of the entity DataFrame




📊 Entity DataFrame:


,Entity Type,Entity Value,Frequency,Query Rows
66,status_check,check,1264,"[39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5..."
46,support_request,help me,880,"[2, 335, 336, 337, 338, 339, 340, 341, 342, 34..."
49,support_request,assistance,722,"[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2..."
67,status_check,to check,613,"[131, 132, 133, 134, 135, 136, 137, 138, 139, ..."
48,support_request,you help,545,"[2, 335, 336, 337, 338, 339, 340, 341, 342, 34..."
...,...,...,...,...
79,invoice_reference,last invoice,1,[1238]
83,support_channel,talk to human,1,[5665]
94,payment_method,card,1,[1198]
99,order_reference,cancel order,1,[726]
